#### Example code
##### Running SQL in Python

In [ ]:
# Here we use an f-string which allows variables to be incorporated into the string.
# The database contains many snapshots in time. These need to be defined for each table
# to avoid returning duplicates

# triple quotation marks allow a string to be defined within the quaotation marks even on new lines
# makes long sql strings much more readable.
# single quotation marks limit the string to a single line.

s1 = f"""
select * from {db1}.cases 
where mojap_snapshot_date = date '{snapshot_date}'
limit 10
"""

# pydbtools read_sql_query function will return the results of the query into a pandas dataframe
simple_results = pydb.read_sql_query(s1)
simple_results

##### Creating temporary tables

In [ ]:
# Here we are creating a temporary table that can be queried later
#  from the __temp__ database
pydb.create_temp_table(s1, "case_table")

# Query the temporary table just created. The database to query is called __temp__, this is
# an alias for a sandbox database that is created for each user. For more details, see
# the pydbtools docs
temp_table = pydb.read_sql_query("select count(*) as count from __temp__.case_table")

temp_table

##### Creating tables in Athena
###### When creating tables in Athena, the underlying data also needs to be written to a folder in S3

In [ ]:
# create path for new table (initial bucket path and sub-folder defined earlier)
test_table_s3_path = os.path.join(fcsq_db_path, "test_table/")

In [ ]:
# create table in Athena FCSQ database, stating the S3 path to store underlying data
test_table = f"""
CREATE TABLE {db2}.test_table WITH
(
    external_location='{test_table_s3_path}'
) AS
SELECT *
FROM __temp__.case_table
"""
# execute the SQL query
_ = pydb.start_query_execution_and_wait(test_table)

##### Deleting tables in Athena

In [ ]:
# Delete all the underlying data stored within the S3 location
if wr.s3.list_objects(test_table_s3_path):
    print("deleting objs")
    wr.s3.delete_objects(test_table_s3_path)

In [ ]:
# drop the table in Athena
drop_test_table = f"""
DROP TABLE {db2}.test_table
"""
# execute the SQL query
_ = pydb.start_query_execution_and_wait(drop_test_table)

In [ ]:
# This also works but not using old versions of pybd. Note the recommended option is the start_query...wait option
pydb.read_sql_query(f"""DROP TABLE {db2}.test_table""")

In [ ]:
# can also embed the code within the brackets rather than creating as a separate object
_ = pydb.start_query_execution_and_wait(f"""DROP TABLE {db2}.test_table""")